In [1]:
from src.guess.guess import ConfigType, Guess, Config
from src.domain.datasets.UnixCommandDataset import UnixCommandDataset  # Register dataset
from src.domain.datasets.ProjectEulerDataset import ProjectEulerDataset

prediction = None

config = Config(ConfigType.QWEN_X862ARM64.get_path())
guess = Guess(config=config)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-22 01:23:37,324 - src.domain.models.QwenModel - INFO - Initializing QwenModel with ahmedheakl/asm2asm_1.5b_armv8_o0 on device mps


Loading Qwen model: ahmedheakl/asm2asm_1.5b_armv8_o0


2025-04-22 01:23:40,641 - QwenModel - INFO - Initialized QwenModel on device: mps:0
2025-04-22 01:23:40,642 - src.domain.models.QwenModel - INFO - Model initialization completed in 3.32 seconds


Source JSONL: data/processed/X86/UnixCommands_x86.jsonl
Target JSONL: data/processed/ARM64/UnixCommands_arm.jsonl
Loading file: data/processed/X86/UnixCommands_x86.jsonl
Loaded 11 entries from data/processed/X86/UnixCommands_x86.jsonl
Loading file: data/processed/ARM64/UnixCommands_arm.jsonl
Loaded 11 entries from data/processed/ARM64/UnixCommands_arm.jsonl


In [2]:
test_instance = guess.data_loader.iter().__next__()
print(test_instance)


2025-04-22 01:23:40,647 - src.helpers.dataset - INFO - Loaded 11 instances
2025-04-22 01:23:40,648 - src.helpers.data_loader - INFO - Loaded 11 samples from dataset


Source entries: 11
Target entries: 11
Creating instance 0 with source key x86 and target key arm
Source entry keys: dict_keys(['source', 'x86', 'x86_output', 'x86_verbose'])
Target entry keys: dict_keys(['source', 'arm', 'arm_output', 'arm_verbose'])
Creating instance 1 with source key x86 and target key arm
Source entry keys: dict_keys(['source', 'x86', 'x86_output', 'x86_verbose'])
Target entry keys: dict_keys(['source', 'arm', 'arm_output', 'arm_verbose'])
Creating instance 2 with source key x86 and target key arm
Source entry keys: dict_keys(['source', 'x86', 'x86_output', 'x86_verbose'])
Target entry keys: dict_keys(['source', 'arm', 'arm_output', 'arm_verbose'])
Creating instance 3 with source key x86 and target key arm
Source entry keys: dict_keys(['source', 'x86', 'x86_output', 'x86_verbose'])
Target entry keys: dict_keys(['source', 'arm', 'arm_output', 'arm_verbose'])
Creating instance 4 with source key x86 and target key arm
Source entry keys: dict_keys(['source', 'x86', 'x86

In [ ]:
test_pred = guess.model.predict(test_instance, guess.inference_cfg)

2025-04-22 01:23:40,657 - src.domain.models.QwenModel - INFO - Prompt preparation completed in 0.01 seconds
2025-04-22 01:23:40,661 - src.domain.models.QwenModel - INFO - Tokenization completed in 0.00 seconds
2025-04-22 01:23:40,662 - src.domain.models.QwenModel - INFO - Starting inference with temperature=0.7, max_length=4096
2025-04-22 01:23:40,662 - src.domain.models.QwenModel - INFO - Input shape: torch.Size([1, 750])
`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [ ]:
from src.sketch.sketch import Sketch

sketch = Sketch(config, guess.model)

In [ ]:
import pandas as pd
from IPython.display import HTML, display

def html_escape(text):
    # Convert special characters to HTML-safe versions
    return (text.replace("&", "&amp;")
                .replace("<", "&lt;")
                .replace(">", "&gt;")
                .replace("\n", "<br>")
                .replace("\t", "&nbsp;&nbsp;&nbsp;&nbsp;"))  # 4 spaces for a tab

def display_blocks(blocks, pred):
    data = []

    for block in blocks:
        source_text = sketch.model.tokenizer.decode(
            pred.source[0][block.source_start:block.source_end]
        )
        pred_text = sketch.model.tokenizer.decode(
            pred.pred[0][block.pred_start:block.pred_end]
        )
        data.append({
            'SOURCE': html_escape(source_text),
            'PRED': html_escape(pred_text)
        })

    df = pd.DataFrame(data)

    styles = """
    <style>
    table {
        table-layout: auto;
        word-wrap: break-word;
    }
    td {
        white-space: normal !important;
        font-family: monospace;
        vertical-align: top;
    }
    </style>
    """
    display(HTML(styles + df.to_html(escape=False)))


In [ ]:
# test_pred = predictions_test['UnixCommands/cat']
line_mappings = sketch.map_predicted_lines_to_source_lines(
    test_pred.source,
    test_pred.pred,
    test_pred.alignments
)
print(line_mappings)
blocks = sketch.extract_pure_instruction_blocks(
    test_pred.source, test_pred.pred, line_mappings
)

print(blocks)

display_blocks(blocks, test_pred)

In [ ]:
results = sketch.sketch({"test_pred": test_pred})

for pred_result, sketch_result in results:
    print(pred_result.instance_id)
    print(f"Ratio of invalid_blocks: {len(sketch_result.invalid_blocks)/len(sketch_result.total_blocks)}")
    print(f"Ratio of non_equivalent_blocks: {len(sketch_result.non_equivalent_blocks)/len(sketch_result.total_blocks)}")